<a href="https://colab.research.google.com/github/jisang93/SNS_Project/blob/master/Insta_Crwaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import requests, re, json, urllib, datetime
import pandas as pd
from bs4 import BeautifulSoup

#search = '%EC%9D%B4%EC%8A%B9%EA%B8%B0'
#def insta(query):
#search = urllib.parse.quote(query)

# 헤더 저장
headers = {
            'cookie' : 'ig_did=1DB997A1-85F5-410B-A44B-DB3A31DE9BF2; mid=XxLtBgALAAHy9DlenU_IjVp8c-vI; csrftoken=w8fk3ZvDCg3GeMsgOGjj4c7C1J1CDDEv; ds_user_id=2164165712; sessionid=2164165712%3A0i2iNaTHNVLjRS%3A25; shbid=980; shbts=1597821443.3613222; rur=VLL; urlgen="{\"222.107.238.125\": 4766}:1k90Xa:Xh5aOhIn5p2sNhAMjIh4OesdfJk"',
            'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
            }
# 해시 태그 설정
tags = "20대"

res = requests.get('https://www.instagram.com/explore/tags/{}/?__a=1'.format(tags), headers = headers)
res.encoding = 'utf-8'
js = json.loads(res.text)

In [30]:
user_url = []
for i in js['graphql']['hashtag']['edge_hashtag_to_media']['edges']:
    # user 이름으로 접속할 경우 게시글이 다른 게시물로 넘어감
    
    user_url.append('https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"{}"%2C"first"%3A12%7D'.format(i['node']['owner']['id']))

user_url[:5]

['https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"38536790088"%2C"first"%3A12%7D',
 'https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"38536790088"%2C"first"%3A12%7D',
 'https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"38536790088"%2C"first"%3A12%7D',
 'https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"38536790088"%2C"first"%3A12%7D',
 'https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"38536790088"%2C"first"%3A12%7D']

In [48]:
#post_user 형태 =  {"user_ID" : {"user_info" : "user_name", "post_content" : [{"post_code": "post_text", "post_location", "post_like"},
#                                                                             {"post_code": "post_text", "post_location", "post_like"}]}}}                                                
import time

post_user = {}
crawl_count = 0

for url in user_url:
    url = url[:-3]
    url_end = '%7D'
    res = requests.get(url+url_end, headers=headers)
    print(url)
    res.encoding = 'utf-8'
    js = json.loads(res.text)
    while True:
        if js['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page'] == True: # 다음 페이지가 존재할 경우
            new_url = url + '%2C"after"%3A"{}"'.format(js['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor'])+ url_end
            print(new_url)
            res1 = requests.get(new_url)
            res1.encoding='utf-8'
            js = json.loads(res1.text)
            new_post = {}
            for i in range(len(js['data']['user']['edge_owner_to_timeline_media']['edges'])):
                # 200회 크롤링 후 5분 휴식
                crawl_count += 1
                if crawl_count > 1000:
                    time.sleep(200)
                # 인스타에서 부여한 유저 ID
                user_id = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['owner']['id']
                new_post = {}
                if user_id not in new_post.keys() :
                    new_post[user_id] = {}      # 유저 아이디의 딕셔너리가 없으면 추가해줌
                # 유저명
                new_post[user_id]['user_name'] = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['owner']['username']
                # 글 작성 지역
                if js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['location'] != None:
                    location = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['location']['name']
                else:
                    location = None
                # 유저 작성 글 정보
                if 'post_content' not in new_post[user_id].keys():
                    new_post[user_id]['post_content'] = []
                new_post[user_id]['post_content'].append({'post_code' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['shortcode'],    # 글 코드
                                                          'post_text' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_caption']['edges'][0]['node']['text'],      # 글 내용
                                                          'post_time' : (datetime.datetime.fromtimestamp(int(js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['taken_at_timestamp']))).strftime('%Y-%m-%d %H:%M:%S'),   # 글 작성일자
                                                          'post_location' : location, # 글 작성 장소
                                                          'post_like' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_preview_like']['count']})     # 좋아요 개수
                if user_id not in post_user.keys() :
                    post_user[user_id] = new_post[user_id]      # 사용자 ID가 존재하지 않을 경우 딕셔너리를 새로 만듦
                else :
                    post_user[user_id]['post_content'] = post_user[user_id]['post_content'] + new_post[user_id]['post_content'] # 존재할 경우 글 정보만 추가
                # 크롤링 진행 상황 출력
                if craw_count % 100 == 0:
                    print("크롤링 횟수 : " + crawl_count)

        else:
            res1 = requests.get(new_url)
            res1.encoding='utf-8'
            js = json.loads(res1.text)
            new_post = {}
            for i in range(len(js['data']['user']['edge_owner_to_timeline_media']['edges'])):
                crawl_count += 1
                if crawl_count > 1000:
                    time.sleep(200)
                user_id = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['owner']['id'] # 유저 아이디 기준으로 딕셔너리 생성
                new_post = {}
                if user_id not in new_post.keys() :
                    new_post[user_id] = {}
                new_post[user_id]['user_name'] = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['owner']['username']
                if js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['location'] != None:
                    location = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['location']['name']
                else:
                    location = None
                if 'post_content' not in new_post[user_id].keys():
                    new_post[user_id]['post_content'] = []
                new_post[user_id]['post_content'].append({'post_code' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['shortcode'],
                                                        'post_text' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_caption']['edges'][0]['node']['text'], 
                                                        'post_time' : (datetime.datetime.fromtimestamp(int(js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['taken_at_timestamp']))).strftime('%Y-%m-%d %H:%M:%S'),
                                                        'post_location' : location,
                                                        'post_like' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_preview_like']['count']})
                if user_id not in post_user.keys() :
                    post_user[user_id] = new_post[user_id]
                else :
                    post_user[user_id]['post_content'] = post_user[user_id]['post_content'] + new_post[user_id]['post_content']
                # 크롤링 진행 상황 출력
                if craw_count % 100 == 0:
                    print("크롤링 횟수 : " + crawl_count)
            break

post_user

https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"38536790088"%2C"first"%3A12
https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"38536790088"%2C"first"%3A12%2C"after"%3A"QVFDSWxKYjY5UzRmR0Z4WUhlNnZUcU9JU1FuODJiVW5OSERJWXhpT0lrb3dQcXNqMnk5RW9IWVMwMGE1bVZkSjh2ZWVHb2ZMVXB4VmM0Q2pTUHNMd1Zxbw=="%7D
https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"38536790088"%2C"first"%3A12%2C"after"%3A"QVFEZHM1SWw3X3R5V3p0eHNHZUtmbGlONXdZbWpQOHo1YlRQWlhCN05EMHhkUlN4Y1hPWks5WDg2T3VZaVBpcFR2bHB2cUZVNEM4SEtjbzhpdlVnd1VTWg=="%7D
https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"38536790088"%2C"first"%3A12%2C"after"%3A"QVFDTHRDekpHZDNIY2o4Q0xGV1MwbWZtVW9IYVZHMDFWV3hYY1BSVXQ3YWNxamJuaVNmbnJnNDlmQXF2aHRDOHoxVHRVeTdGOG1abHlUcExpbXZOenpPRQ=="%7D
https://www.instagram.com/graphql/query/?query_has

KeyboardInterrupt: ignored

In [49]:
post_user

{'38536790088': {'post_content': [{'post_code': 'CEQX7EspN_u',
    'post_like': 0,
    'post_location': None,
    'post_text': '100%본사정품 최신제조 최신상품만 취급하여 판매하는 정직한 코스메틱 뷰티몰🌷\n⭐️할인과 이벤트중⭐️\n맘편히 정직하고 저렴한 가격으로 쇼핑하세요^^\n.\n.\n.\n#아모레퍼시픽 #설화수 #헤라 #리리코스 #바이탈뷰티 #건강기능식품 #피부관리 #건강관리 #20대 #20대후반 #30대 #40대 #육아맘 #맘스타그램 #주부스타그램 #줌마스타그램 #직장인 #뷰티스타그램  #선물세트추천 #좋반 #맞팔 #선팔 #맞팔선팔 #셀카 #소통 #좋테  #이벤트 #출근 #사랑의선물 #셀카',
    'post_time': '2020-08-24 02:47:56'},
   {'post_code': 'CEQX2LlpiG7',
    'post_like': 7,
    'post_location': None,
    'post_text': '100%본사정품 최신제조 최신상품만 취급하여 판매하는 정직한 코스메틱 뷰티몰🌷\n⭐️할인과 이벤트중⭐️\n맘편히 정직하고 저렴한 가격으로 쇼핑하세요^^\n.\n.\n.\n#아모레퍼시픽 #설화수 #헤라 #리리코스 #바이탈ᄇ

In [51]:
def insta_crwal(js):
    for i in range(len(js['data']['user']['edge_owner_to_timeline_media']['edges'])):
        crawl_count += 1
        if crawl_count > 1000:
            time.sleep(200)
        user_id = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['owner']['id'] # 유저 아이디 기준으로 딕셔너리 생성
        new_post = {}
        if user_id not in new_post.keys() :
            new_post[user_id] = {}
        new_post[user_id]['user_name'] = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['owner']['username']
        if js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['location'] != None:
            location = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['location']['name']
        else:
            location = None
        if 'post_content' not in new_post[user_id].keys():
            new_post[user_id]['post_content'] = []
        new_post[user_id]['post_content'].append({'post_code' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['shortcode'],
                                                'post_text' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_caption']['edges'][0]['node']['text'], 
                                                'post_time' : (datetime.datetime.fromtimestamp(int(js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['taken_at_timestamp']))).strftime('%Y-%m-%d %H:%M:%S'),
                                                'post_location' : location,
                                                'post_like' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_preview_like']['count']})
        if user_id not in post_user.keys() :
            post_user[user_id] = new_post[user_id]
        else :
            post_user[user_id]['post_content'] = post_user[user_id]['post_content'] + new_post[user_id]['post_content']
        # 크롤링 진행 상황 출력
        if craw_count % 100 == 0:
            print("크롤링 횟수 : " + crawl_count)

        return post_user